In [ ]:
# Air drawing App

import cv2
import numpy as np
import mediapipe as mp

cap = cv2.VideoCapture(0)
mp_hands = mp.solutions.hands
mp_draw = mp.solutions.drawing_utils
hands = mp_hands.Hands(min_detection_confidence=0.7,min_tracking_confidence=0.7)


canvas = None
prev_x,prev_y = 0,0

def finger_up(lms):
    fingers = []
    fingers.append(1 if lms[4][0] > lms[3][0] else 0)
    for tip in [8,12,16,20]:
        fingers.append(1 if lms[tip][1] < lms[tip-2][1] else 0)
    return fingers


        

while cap.isOpened():
    r,f = cap.read()
    if r==False:
        break
    f = cv2.flip(f,1)
    rgb = cv2.cvtColor(f,cv2.COLOR_BGR2RGB)
    res = hands.process(rgb)
    h,w,_ = f.shape

    if canvas is None:
        canvas = np.zeros_like(f)


    lms = []
    if res.multi_hand_landmarks:
        for hand_landmarks in res.multi_hand_landmarks:
            for id,lm in enumerate(hand_landmarks.landmark):
                cx,cy = int(lm.x*w),int(lm.y*h)
                lms.append((cx,cy))
            mp_draw.draw_landmarks(f,hand_landmarks,mp_hands.HAND_CONNECTIONS)
    
    if(len(lms)>=21):
        fingers = finger_up(lms)

        if fingers[1]==1 and all(f==0 for i,f in enumerate(fingers) if i!=1):
            x,y = lms[8]
            if prev_x==0 and prev_y==0:
                prev_x,prev_y=x,y
            cv2.line(canvas,(prev_x,prev_y),(x,y),(255,0,0),5)
            prev_x,prev_y=x,y
        else:
            prev_x,prev_y=0,0

        if fingers==[1,1,1,1,1]:
            cv2.putText(f,"Clearing Canvas ....",(10,70),cv2.FONT_HERSHEY_COMPLEX_SMALL,0.5,(0,0,255),2)
            canvas = np.zeros_like(f)
            

    gray_canvas = cv2.cvtColor(canvas,cv2.COLOR_BGR2GRAY)
    _,inv = cv2.threshold(gray_canvas,50,255,cv2.THRESH_BINARY_INV)
    inv = cv2.cvtColor(inv,cv2.COLOR_GRAY2BGR)
    f = cv2.bitwise_and(f,inv)
    f = cv2.bitwise_or(f,canvas)

    cv2.putText(f,"Air Drawing - Index finger to draw.",(10,30),cv2.FONT_HERSHEY_COMPLEX_SMALL,0.8,(0,255,0),2)
    cv2.imshow("Air Drawing app",f)
    if cv2.waitKey(25) & 0xff == ord('d'):
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
import cv2
import numpy as np
import mediapipe as mp

cap = cv2.VideoCapture(0)
mp_hands = mp.solutions.hands
mp_draw = mp.solutions.drawing_utils
hands = mp_hands.Hands(min_detection_confidence=0.8,min_tracking_confidence=0.8)

canvas = None
draw_color = (0,255,0)
brush_thickness = 7
erase_thickness = 40
color_selected="red"

px,py = 0,0

def finger_ups(lm):
    finger_tips = [4,8,12,16,20]
    fingers = []

    if lm.landmark[4].x < lm.landmark[3].x:
        fingers.append(1)
    else:
        fingers.append(0)

    for tip in finger_tips[1:]:
        if lm.landmark[tip].y < lm.landmark[tip-2].y:
            fingers.append(1)
        else:
            fingers.append(0)
    return fingers

while cap.isOpened():
    r,f = cap.read()
    # f = f.reshape(1000,1000)
    if r==False:
        break
    f = cv2.flip(f,1)
    cv2.rectangle(f,(0,0),(100,50),(0,0,255),-1)
    cv2.rectangle(f,(105,0),(220,50),(0,255,0),-1)
    cv2.rectangle(f,(225,0),(325,50),(255,0,0),-1)

    cv2.putText(f,"RED",(20,30),cv2.FONT_HERSHEY_COMPLEX_SMALL,1,(255,255,255),2)
    cv2.putText(f,"GREEN",(120,30),cv2.FONT_HERSHEY_COMPLEX_SMALL,1,(255,255,255),2)
    cv2.putText(f,"BLUE",(250,30),cv2.FONT_HERSHEY_COMPLEX_SMALL,1,(255,255,255),2)
    if canvas is None:
        canvas = np.zeros_like(f)

    rgb = cv2.cvtColor(f,cv2.COLOR_BGR2RGB)
    res = hands.process(rgb)

    if res.multi_hand_landmarks:
        for landmarks in res.multi_hand_landmarks:
            mp_draw.draw_landmarks(f,landmarks,mp_hands.HAND_CONNECTIONS)
            lm = landmarks.landmark
            h,w,c = f.shape
            x1,y1 =  int(lm[8].x*w),int(lm[8].y*h)
            x2,y2 =  int(lm[12].x*w),int(lm[12].y*h)

            if y1<100:
                if 0<x1<100:
                    draw_color=(0,0,255)
                    color_selected = "red"
                elif 100<x1<200:
                    draw_color=(0,255,0)
                    color_selected = "green"
                elif 200<x1<300:
                    draw_color=(255,0,0)
                    color_selected = "blue"

            fingers = finger_ups(landmarks)
            drawing_mode = False
            eraseing_mode = False

            if fingers[1] == 1 and fingers[2]==0:
                drawing_mode=True
                eraseing_mode=False
                
            elif fingers[1]==1 and fingers[2]==1:
                drawing_mode=False
                eraseing_mode=True
            else:
                drawing_mode=False
                eraseing_mode=False
                px,py = 0,0

            if drawing_mode:
                cv2.circle(f,(x1,x2),10,draw_color,1)
                if px==0 and py == 0:
                    px,py = x1,y1
                cv2.line(canvas,(px,py),(x1,y1),draw_color,brush_thickness)
                px,py = x1,y1
            if eraseing_mode:
                # cv2.circle(f,(x1,x2),15,(0,0,0),cv2.FILLED)
                if px==0 and py == 0:
                    px,py = x1,y1
                cv2.line(canvas,(px,py),(x1,y1),(0,0,0),erase_thickness)
                px,py = x1,y1

            if sum(fingers)==5:
                cv2.putText(f,"Clearing Canva....",(10,70),cv2.FONT_HERSHEY_COMPLEX_SMALL,1,(0,0,255),3)
                canvas = np.zeros_like(f)
    
    gray_canvas = cv2.cvtColor(canvas,cv2.COLOR_BGR2GRAY)
    _,inv_canvas = cv2.threshold(gray_canvas,50,255,cv2.THRESH_BINARY_INV)
    inv_canvas = cv2.cvtColor(inv_canvas,cv2.COLOR_GRAY2BGR)
    img = cv2.bitwise_and(f,inv_canvas)
    img = cv2.bitwise_or(f,canvas)
            

    cv2.putText(img,f"Brush : {color_selected}",(10,130),cv2.FONT_HERSHEY_COMPLEX_SMALL,1,draw_color,2)
    cv2.imshow("Air Drawing",img)
    if cv2.waitKey(25) & 0xff == ord('d'):
        break

cap.release()
cv2.destroyAllWindows()